# Querying Faculty Demographic Data from IPEDS

This script queries [US IPEDS MS Access files](https://nces.ed.gov/ipeds/use-the-data/download-access-database) and generates datasets for analyzing diversity among tenured faculty.

Requirements:
* [mdb-tools](https://github.com/mdbtools/mdbtools)

Tables of note:
* `Directory Information`: contains directory information with name, address, city, state, zipcode, etc
* `Response Status`: did the institution respond, or were the data imputed?
* `Full-time instructional/research/public service  staff, by faculty and tenure status, academic rank, race/ethnicity, and gender (Degree-granting institutions): Fall YYYY`: Multiple records per institution. 
  * CAUTION:  Reporting Human resource data by race/ethnicity and gender is optional in even-numbered years, so many institutions will not have data.
  * Beginning with 2016 reporting human resource data by race/ethnicity and gender is mandatory annually.
* New hires by occupational category, race/ethnicity, and gender (Degree-granting institutions):  Fall YYYY: Multiple records per institution

Other data used:
* CCIHE2018-PublicData.xlsx: [Carnegie Classification Data](https://carnegieclassifications.iu.edu/downloads.php)
* mrc_table10.csv: [College Level Characteristics from the Opportunity Insights College Scorecard](https://opportunityinsights.org/wp-content/uploads/2018/04/Codebook-MRC-Table-10.pdf)


# Configuration

In [52]:
## LOAD LIBRARIES
from plotnine import ggplot, aes, geom_line
import numpy as np
import pandas as pd


import statsmodels.api as sm
import statsmodels.formula.api as smf

from collections import defaultdict, Counter
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import pyplot
import glob, os, sys, subprocess # for accessing mdb-tools
#import simplejson as json
import csv
import codecs
import datetime
import copy
import re #for string matching

##load simplejson to fix error
##!pip install simplejson as json
import subprocess
subprocess.check_call(['pip', 'install', 'simplejson==3.19.3'])

data_path = r"C:\Users\jnajab.AD\OneDrive - AAAS\Documents\SEAChange\Faculty Parity Project\US-Faculty-Diversity-Projections-main\data\IPEDS_DATA_FOLDER"

## data_path = "C:\\Users\\jnajab.AD\\OneDrive - AAAS\\Documents\\SEAChange\\Faculty Parity Project\\US-Faculty-Diversity-Projections-main\\data\\IPEDS_DATA_FOLDER\\"

In [53]:
## Define Table Names - hand checked from IPEDSYYYYYYTablesDoc.xlsx for each year
table_info = {
    "2004-05":{
        "directory":"HD2004",
        "faculty_demographics":"S2004_F",
        "rank_columns": "2004" # which rank categories 
    },
    "2005-06":{
        "directory":"HD2005",
        "faculty_demographics":"S2005_F",
        "rank_columns": "2004" # which rank categories 
    },
    "2006-07":{
        "directory":"HD2006",
        "faculty_demographics":"S2006_F",
        "rank_columns": "2004" # which rank categories 
    },
    "2007-08":{
        "directory":"HD2007",
        "faculty_demographics":"S2007_F",
        "rank_columns": "2004" # which rank categories 
    },
    "2008-09":{
        "directory":"HD2008",
        "faculty_demographics":"S2008_F",
        "rank_columns": "2004" # which rank categories 
    },
    "2009-10":{
        "directory":"HD2009",
        "faculty_demographics":"S2009_F",
        "rank_columns": "2004" # which rank categories 
    },
    "2010-11":{
        "directory":"HD2010",
        "faculty_demographics":"S2010_F",
        "rank_columns": "2004" # which rank categories 
    },
    "2011-12":{
        "directory":"HD2011",
        "faculty_demographics":"S2011_F",
        "rank_columns": "2004" # which rank categories 
    },
    "2012-13":{
        "directory":"HD2012",
        "faculty_demographics":"S2012_IS",
        "rank_columns": "2012"
    },
    "2013-14":{
        "directory":"HD2013",
        "faculty_demographics":"S2013_IS",
        "rank_columns": "2012"
    },
    "2014-15":{
        "directory":"HD2014",
        "faculty_demographics":"S2014_IS",
        "rank_columns": "2012"
    },
    "2015-16":{
        "directory":"HD2015",
        "faculty_demographics":"S2015_IS",
        "rank_columns": "2012"
    },
    "2016-17":{
        "directory":"HD2016",
        "faculty_demographics":"S2016_IS",
        "rank_columns": "2012"
    },
    "2017-18":{
        "directory":"HD2017",
        "faculty_demographics":"S2017_IS",
        "rank_columns": "2012"
    },
    "2018-19":{
        "directory":"HD2018",
        "faculty_demographics":"S2018_IS",
        "rank_columns": "2012"
    },
    "2019-20":{
        "directory":"HD2019",
        "faculty_demographics":"S2019_IS",
        "rank_columns": "2012"
    },
    "2020-21":{
        "directory":"HD2020",
        "faculty_demographics":"S2020_IS",
        "rank_columns": "2012"
    },
        "2021-22":{
        "directory":"HD2021",
        "faculty_demographics":"S2021_IS",
        "rank_columns": "2012"
    },
        "2022-23":{
        "directory":"HD2022",
        "faculty_demographics":"S2022_IS",
        "rank_columns": "2012"
    }
}

# Load Files

In [57]:
## folders that we need to store but which
## shouldn't be included in the data. List
## folders representing provisional data for which
## we now have final data

folders_to_omit = ["IPEDS_2019-20_Provisional"]

folders = [x for x in glob.glob(data_path + "*") if x.find("zip")==-1 
#folders = [x for x in glob.glob(IPEDS_DATA_FOLDER + "*") if x.find("zip")==-1 

           and x.find("csv")==-1 and x.find("outputs")==-1 
           and x.find("xlsx")==-1 and x.find("numbers")==-1]

data_folders = [r"C:\Users\jnajab.AD\OneDrive - AAAS\Documents\SEAChange\Faculty Parity Project\US-Faculty-Diversity-Projections-main\data\IPEDS_DATA_FOLDER"] #######INSERT DATA PATH

for folder in folders:
    include = True
    for o in folders_to_omit:
        if folder.find(o)!=-1:
            include = False
    if(include):
        data_folders.append(folder)
        #IPEDS_DATA_FOLDER.append(folder)
data_folders
#IPEDS_DATA_FOLDER.append(folder)


['C:\\Users\\jnajab.AD\\OneDrive - AAAS\\Documents\\SEAChange\\Faculty Parity Project\\US-Faculty-Diversity-Projections-main\\data\\IPEDS_DATA_FOLDER',
 'C:\\Users\\jnajab.AD\\OneDrive - AAAS\\Documents\\SEAChange\\Faculty Parity Project\\US-Faculty-Diversity-Projections-main\\data\\IPEDS_DATA_FOLDER']

### Load the Institution Directories

In [58]:
## Create institution object 
def institution():
    return {
        "UNITID":None, #ID
        "INSTNM": None, #Name (some overlap),
        "STABBR": None, #State Abbreviation
        
        ## the key for these are academic year, such as "2013-14"
        "directory_year": {},
        "faculty_demographics_year": {}
    }

institutions = defaultdict(institution)


## iterate through each folder / year and fetch metadata
## on the folder names to examine, per year
for folder in data_folders:
#for folder in IPEDS_DATA_FOLDER:
    if folder.find("csv") >-1 or folder.find("xlsx") >-1 or folder.find("outputs")>-1:
        continue
    year = re.match(".*?IPEDS_(.*?)_",folder).groups()[0]
    table = table_info[year]
    
    ## load file
    filename = glob.glob(folder+"/*.accdb")[0]
    directory = subprocess.Popen(['mdb-export', filename, table['directory']], stdout=subprocess.PIPE).communicate()[0].decode()
    
    counter = 0
    for row in csv.DictReader(directory.split("\n")):
        institutions[row['UNITID']]['UNITID'] = row['UNITID']
        institutions[row['UNITID']]['INSTNM'] = row['INSTNM']
        institutions[row['UNITID']]['STABBR'] = row['STABBR']
        institutions[row['UNITID']]['directory_year'][year] = row
        counter += 1
    print("{0}: Loaded {1} institutions".format(year, counter))

KeyError: 'DATA'

## Load Student Demographic Records (to identify primarily-white institutions)
PWI doesn't have an agreed-on definition.  I found a graduate thesis that labeled an institution a PWI if it was not listed as an HBCU or HSI. I plan to label institutions as a PWI based on the percentage of white students reported in the most recent datafile. Institutions with more than 50% white students will be labeled PWIs.


In [42]:
pct_white_column = "PCTENRWH"

## set up variables for loading the most recent table
folder = [folder for folder in data_folders if folder.find("csv") ==-1 and folder.find("xlsx") ==-1 and folder.find("outputs") == -1][-1]
#folder = [folder for folder in IPEDS_DATA_FOLDER if folder.find("csv") ==-1 and folder.find("xlsx") ==-1 and folder.find("outputs") == -1][-1]
year = re.match(".*?IPEDS_(.*?)_",folder).groups()[0]
#year = re.match("/IPEDS_DATA_FOLDER/.*?IPEDS_(.*?)_",folder).groups()[0]

filename = glob.glob(folder+"/*.accdb")[0]
deriv_variable_tablename = "DRVEF" + year[0:4]

## load raw table text into table_data
table_data = subprocess.Popen(['mdb-export', filename, deriv_variable_tablename], stdout=subprocess.PIPE).communicate()[0].decode()

## load table_data into a dict for determining primarily-white-institutions
institution_pwi = {}
for row in csv.DictReader(table_data.split("\n")):
    # set UNITID key, since capitalization is inconsistent between years
    unitid_key = "UNITID"
    if(unitid_key not in row.keys()):
        unitid_key = "UnitID"
    unitid = row[unitid_key]
    
    if(pct_white_column in row.keys()):
        pct_white = int(row[pct_white_column])
    else:
        pct_white = None
    institution_pwi[unitid] = pct_white


IndexError: list index out of range

## Load the Faculty Demographics Records

In [29]:
for folder in data_folders:
    if folder.find("csv") >-1 or folder.find("xlsx") >-1 or folder.find("outputs")>-1:
        continue
    year = re.match(".*?IPEDS_(.*?)_",folder).groups()[0]
    table = table_info[year]
    
    ## load file
    filename = glob.glob(folder+"/*.accdb")[0]
    table_data = subprocess.Popen(['mdb-export', filename, table['faculty_demographics']], stdout=subprocess.PIPE).communicate()[0].decode()
    
    
    counter = 0
    for row in csv.DictReader(table_data.split("\n")):
        
        # set UNITID key, since capitalization is inconsistent between years
        unitid_key = "UNITID"
        if(unitid_key not in row.keys()):
            unitid_key = "UnitID"
        unitid = row[unitid_key]
        
        if year not in institutions[unitid]['faculty_demographics_year'].keys():
            institutions[unitid]['faculty_demographics_year'][year] = []
        institutions[unitid]['faculty_demographics_year'][year].append(row)
        counter += 1
        
    print("{0}: Loaded {1} faculty demographics records".format(year, counter))

### Convert IPEDS columns to human-readable columns

2004-2005 codes:
* **ARANK:** 
  * 22: *Total full-time faculty*
  * 1: Total full-time faculty, Tenured, Professors 
  * 2: Total full-time faculty, Tenured, Associate professors
  * 3: Total full-time faculty, Tenured, Assistant professors
  * <del> 4: Total full-time faculty, Tenured, Instructors</del>
  * <del> 5: Total full-time faculty, Tenured, Lecturers</del>
  * 6: <del> Total full-time faculty, Tenured, No academic rank</del>
  * 7: *Total full-time faculty, Tenured total*
  * 8: Total full-time faculty, Non-tenured on tenure track, Professors
  * 9: Total full-time faculty, Non-tenured on tenure track, Associate professors
  * 10: Total full-time faculty, Non-tenured on tenure track, Assistant professors
  * <del> 11: Total full-time faculty, Non-tenured on tenure track, Instructors</del>
  * <del> 12: Total full-time faculty, Non-tenured on tenure track, Lecturers</del>
  * 13: <del> Total full-time faculty, Non-tenured on tenure track, No academic rank</del>
  * 14: *Total full-time faculty, Non-tenured on tenure track total*

* **Staff Columns:**
    * STAFF19	American Indian or Alaska Native total
    * STAFF20	Asian or Pacific Islander total
    * STAFF21	Hispanic total
    * STAFF22	White non-Hispanic total
    * STAFF01	Nonresident alien men
    * STAFF02	Nonresident alien women
    * STAFF03	Black non-Hispanic men
    * STAFF04	Black non-Hispanic women
    * STAFF05	American Indian or Alaska Native men
    * STAFF06	American Indian or Alaska Native women
    * STAFF07	Asian or Pacific Islander men
    * STAFF08	Asian or Pacific Islander women
    * STAFF09	Hispanic men
    * STAFF10	Hispanic women
    * STAFF11	White non-Hispanic men
    * STAFF12	White non-Hispanic women
    * STAFF13	Race/ethnicity unknown men
    * STAFF14	Race/ethnicity unknown women
    * STAFF15	Grand total men
    * STAFF16	Grand total women
    * STAFF17	Nonresident alien total
    * STAFF18	Black non-Hispanic  total
    * STAFF23	Race/ethnicity unknown total
    * STAFF24	Grand total


### Metadata for Querying Records and Merging Columns

In [30]:
## since the rows change and the meaning of the integers change
## at certain points, I have created these methods and placed
## them into a dict so the right method can be called for the
## right piece of information, depending on what year the record is from

def rank_2004_total_tenured(row):
    return row['ARANK']=='7'

## non_tenured is different from untenured tenure-track
def rank_2004_total_non_tenured(row):
    return row['ARANK']=='14'

def rank_2004_tenured_full(row):
    return row['ARANK']=='1'
def rank_2004_tenured_associate(row):
    return row['ARANK']=='2'
def rank_2004_tenured_assistant(row):
    return row['ARANK']=='3'

## non_tenured is different from untenured tenure-track
def rank_2004_non_tenured_full(row):
    return row['ARANK']=='8'
## non_tenured is different from untenured tenure-track
def rank_2004_non_tenured_associate(row):
    return row['ARANK']=='9'
## non_tenured is different from untenured tenure-track
def rank_2004_non_tenured_assistant(row):
    return row['ARANK']=='10'

## return empty list for categories that 
## don't exist in the 2004 data
def rank_2004_no_values(row):
    return []

rank_2004 = {
    "rank_object":           "2004",
    "tenured":         rank_2004_total_tenured,
    "non_tenured":     rank_2004_total_non_tenured,
    "un_tenured":      rank_2004_no_values,

    "tenured_full":          rank_2004_tenured_full,
    "tenured_associate":     rank_2004_tenured_associate,
    "tenured_assistant":     rank_2004_tenured_assistant,
    
    "non_tenured_full":      rank_2004_non_tenured_full,
    "non_tenured_associate": rank_2004_non_tenured_associate,
    "non_tenured_assistant": rank_2004_non_tenured_assistant,
    
    "un_tenured_full":       rank_2004_no_values,
    "un_tenured_associate":  rank_2004_no_values,
    "un_tenured_assistant":  rank_2004_no_values
}

## CREATE A DICT FOR HUMAN READABLE COLUMNS

# different keys different years for nonresidents
# STAFF prefixes are before 2010-11
# HR prefixes are from 2010-11 to 2011-12 
col_keys = {"STAFF19":"American Indian or Alaska Native total",
             "HRAIANT":"American Indian or Alaska Native total",

             "STAFF05":"American Indian or Alaska Native men",
             "HRAIANM":"American Indian or Alaska Native men",

             "STAFF06":"American Indian or Alaska Native women",
             "HRAIANW":"American Indian or Alaska Native women",

            ## Changes to Black or African American in 2010-11
             "STAFF18":"Black non-Hispanic total",             
             "STAFF03":"Black non-Hispanic men",             
             "STAFF04":"Black non-Hispanic women",

             ## Changes to Asian | Native Hawaiian or Other Pacific Islander in 2010-11            
             "STAFF20":"Asian or Pacific Islander total",
             "STAFF07":"Asian or Pacific Islander men",
             "STAFF08":"Asian or Pacific Islander women",

             ## Changes to Hispanic or Latino Men in 2010-11
             "STAFF21":"Hispanic total",
             "STAFF09":"Hispanic men",
             "STAFF10":"Hispanic women",

             ## Changes to White in 2010-11
             "STAFF22":"White non-Hispanic total",
             "STAFF11":"White non-Hispanic men",             
             "STAFF12":"White non-Hispanic women",

             # Race/ethnicity unknown
             "STAFF23":"Race/ethnicity unknown total",
             "HRUNKNT":"Race/ethnicity unknown total",
             
             "STAFF13":"Race/ethnicity unknown men",
             "HRUNKNM":"Race/ethnicity unknown men",
             
             "STAFF14":"Race/ethnicity unknown women",
             "HRUNKNW":"Race/ethnicity unknown women",
             
             # Grand Totals
             "STAFF24":"Grand total",
             "HRTOTLT":"Grand total",

             "STAFF15":"Grand total men",
             "HRTOTLM":"Grand total men",
             
             "STAFF16":"Grand total women",
             "HRTOTLW":"Grand total women",

             # Nonresident alien
             "STAFF17":"Nonresident alien total",
             "HRNRALT":"Nonresident alien total",
             
             "STAFF01":"Nonresident alien men", 
             "HRNRALM":"Nonresident alien men",
             
             "STAFF02":"Nonresident alien women",
             "HRNRALW":"Nonresident alien women",
             
             
             ## Not included datasets before 2010-11
             "HRASIAT":"Asian total",
             "HRASIAM":"Asian men",
             "HRASIAW":"Asian women",
             "HRBKAAT":"Black or African American total",
             "HRBKAAM":"Black or African American men",
             "HRBKAAW":"Black or African American women", 
             "HRHISPT":"Hispanic or Latino total",
             "HRHISPM":"Hispanic or Latino men",
             "HRHISPW":"Hispanic or Latino women",
             "HRNHPIT":"Native Hawaiian or Other Pacific Islander total", 
             "HRNHPIM":"Native Hawaiian or Other Pacific Islander men", 
             "HRNHPIW":"Native Hawaiian or Other Pacific Islander women",
             "HRWHITT":"White total",
             "HRWHITM":"White men",
             "HRWHITW":"White women", 
             "HR2MORT":"Two or more races total",
             "HR2MORM":"Two or more races men",
             "HR2MORW":"Two or more races women"  
            }


### Metadata for Querying Records from 2012-13 through 2018-19¶

In [31]:
def rank_2012_total_tenured(row):
    return row['ARANK']=='0' and row['SISCAT'] == '200'

# un-tenured and on tenure track
def rank_2012_total_un_tenured(row):
    return row['ARANK']=='0' and row['SISCAT'] == '300'

def rank_2012_tenured_full(row):
    return row['ARANK']=='1' and row['SISCAT'] == '201'
def rank_2012_tenured_associate(row):
    return row['ARANK']=='2' and row['SISCAT'] == '202'
def rank_2012_tenured_assistant(row):
    return row['ARANK']=='3' and row['SISCAT'] == '203'
def rank_2012_un_tenured_full(row):
    return row['ARANK']=='1' and row['SISCAT'] == '301'
def rank_2012_un_tenured_associate(row):
    return row['ARANK']=='2' and row['SISCAT'] == '302'
def rank_2012_un_tenured_assistant(row):
    return row['ARANK']=='3' and row['SISCAT'] == '303'

## return empty list for categories that 
## don't exist in the 2012 data and onward
def rank_2012_no_values(row):
    return []


rank_2012 = {
    "rank_object":           "2012",
    "tenured":         rank_2012_total_tenured,
    "non_tenured":     rank_2012_no_values,
    "un_tenured":      rank_2012_total_un_tenured,

    "tenured_full":          rank_2012_tenured_full,
    "tenured_associate":     rank_2012_tenured_associate,
    "tenured_assistant":     rank_2012_tenured_assistant,
        
    "un_tenured_full":       rank_2012_un_tenured_full,
    "un_tenured_associate":  rank_2012_un_tenured_associate,
    "un_tenured_assistant":  rank_2012_un_tenured_assistant,
    
    ## NOTE: these are actually available
    ## but I don't know how well they correspond
    ## to how things were collected in the 2004+ period
    ## so I'm omitting them to avoid confusion
    "non_tenured_full":      rank_2012_no_values,
    "non_tenured_associate": rank_2012_no_values,
    "non_tenured_assistant": rank_2012_no_values

}

### Process Data for a Single Institution

In [32]:
## pull out Cornell for initial prototyping and data validation
# institution_match_string = "Michigan"

# [(x['UNITID'], x['INSTNM']) for x in institutions.values() if x['INSTNM'].find(institution_match_string)!=-1]

#institution = institutions['190415']
#institution_name = "Cornell University"

#institution = institutions['186131']
#institution_name = 'Princeton University'

# institution = institutions['166683']
# institution_name = 'Massachusetts Institute of Technology'

# institution = institutions['170976']
# institution_name = 'University of Michigan-Ann Arbor'

In [33]:
# fetch all rows for an institution. Note: this makes heavy
# use of globals
def rows_for_institution(institution):
    
    ## for a given university, iterate through the rank rows 
    ## and collapse them into a single row with many columns
    all_rows = []
    institution_name = institution['INSTNM']

    for year in institution['faculty_demographics_year'].keys():
        new_row = {}
        new_row['year'] = year
        new_row['UNITID'] = institution['UNITID']
        new_row['INSTNM'] = institution['INSTNM']
        new_row['STABBR'] = institution['STABBR']
        if table_info[year]['rank_columns'] =='2004':
            rank_object = rank_2004
            new_row['schema'] = "2004"
        else:
            rank_object = rank_2012
            new_row['schema'] = "2012"
        
#         ## set the grand totals by looking for totals
#         rows = [x for x in institution['faculty_demographics_year'][year] if x['ARANK'] =='0' and x['SISCAT'] in ['200', '300']]
#         try:
#             tenured_un_tenured_total = np.sum([int(x['STAFF24']) for x in rows])
#         except:
#             tenured_un_tenured_total = np.sum([int(x['HRTOTLT']) for x in rows])
            
#         new_row["tenured_un_tenured_Grand_total"] = tenured_un_tenured_total
        
        for rank in rank_object.keys():
            if(rank =="rank_object"):
                continue
            rows = [x for x in institution['faculty_demographics_year'][year] if rank_object[rank](x)]
            for key, new_key in col_keys.items():
                full_key = rank + "_" + new_key.replace(" ","_").replace("/","_")
                if(len(rows)>0):
                    if(key in rows[0].keys()):
                        try:
                            new_row[full_key] = int(rows[0][key])
                        except:
                            new_row[full_key] = None # empty column
                    elif(full_key not in new_row.keys()):
                        new_row[full_key] = None
                else:
                    new_row[full_key] = None
        
        tenured_un_tenured_grand_total = 0
        if(new_row['non_tenured_Grand_total'] is not None):
            tenured_un_tenured_grand_total += int(new_row['non_tenured_Grand_total'])
        if(new_row['un_tenured_Grand_total'] is not None): 
            tenured_un_tenured_grand_total += int(new_row['un_tenured_Grand_total'])
        if(new_row['tenured_Grand_total'] is not None): 
            tenured_un_tenured_grand_total += int(new_row['tenured_Grand_total'])
            
        new_row['tenured_un_tenured_Grand_total'] = tenured_un_tenured_grand_total
        
        all_rows.append(new_row)

    print("Created {0} rows (years) for {1}".format(len(all_rows), institution_name))
    return all_rows

In [34]:
#institutions['100654']['faculty_demographics_year']['2004-05']

# Fetch longitudinal data for all institutions

In [35]:
all_institution_records = []
included_institutions   = []

for key, institution in institutions.items():
    all_records = rows_for_institution(institution)
    
    if(len(all_records)>0):
        all_institution_records += all_records
        included_institutions.append({"UNITID":all_records[0]['UNITID'],
                                      "INSTNM":all_records[0]['INSTNM'], 
                                      "YEARS": len(all_records)})

In [36]:
## List included institutions and how many rows were includedall_institution_records = []
included_institutions   = []

for key, institution in institutions.items():
    all_records = rows_for_institution(institution)
    
    if(len(all_records)>0):
        all_institution_records += all_records
        included_institutions.append({"UNITID":all_records[0]['UNITID'],
                                      "INSTNM":all_records[0]['INSTNM'], 
                                      "YEARS": len(all_records)})
#included_institutions

### Validate Sums for institution Records

In [37]:
## if there are no errors, then the records add up
for record in all_institution_records:
    if(record['tenured_Grand_total_men'] is not None and
       record['tenured_Grand_total_women'] is not None):
        
        assert record['tenured_Grand_total_men'] + record['tenured_Grand_total_women'] == record['tenured_Grand_total']

        if(record['non_tenured_Grand_total'] is not None):
            assert record['tenured_un_tenured_Grand_total'] == record['non_tenured_Grand_total'] + record['tenured_Grand_total'] 
            assert record['non_tenured_Grand_total_men'] + record['non_tenured_Grand_total_women'] == record['non_tenured_Grand_total']
        elif(record['un_tenured_Grand_total'] is not None): 
            assert record['tenured_un_tenured_Grand_total'] == record['un_tenured_Grand_total'] + record['tenured_Grand_total'] 
            assert record['un_tenured_Grand_total_men'] + record['un_tenured_Grand_total_women'] == record['un_tenured_Grand_total']

print("Validated successfully")

Validated successfully


# Load and Merge Carnegie Categories

* [Documentation on Carnegie Classification](https://carnegieclassifications.iu.edu/classification_descriptions/basic.php) 
* Column explanations and data codes are available in the data_path folder in `CCIHE2018-PublicData.xlsx`

#### Meaning of the BASIC2018 codes:
* 1	Associate's Colleges: High Transfer-High Traditional
* 2	Associate's Colleges: High Transfer-Mixed Traditional/Nontraditional
* 3	Associate's Colleges: High Transfer-High Nontraditional
* 4	Associate's Colleges: Mixed Transfer/Career & Technical-High Traditional
* 5	Associate's Colleges: Mixed Transfer/Career & Technical-Mixed Traditional/Nontraditional
* 6	Associate's Colleges: Mixed Transfer/Career & Technical-High Nontraditional
* 7	Associate's Colleges: High Career & Technical-High Traditional
* 8	Associate's Colleges: High Career & Technical-Mixed Traditional/Nontraditional
* 9	Associate's Colleges: High Career & Technical-High Nontraditional
* 10	Special Focus Two-Year: Health Professions
* 11	Special Focus Two-Year: Technical Professions
* 12	Special Focus Two-Year: Arts & Design
* 13	Special Focus Two-Year: Other Fields
* 14	Baccalaureate/Associate's Colleges: Associate's Dominant
* 15	Doctoral Universities: Very High Research Activity
* 16	Doctoral Universities: High Research Activity
* 17	Doctoral/Professional Universities
* 18	Master's Colleges & Universities: Larger Programs
* 19	Master's Colleges & Universities: Medium Programs
* 20	Master's Colleges & Universities: Small Programs
* 21	Baccalaureate Colleges: Arts & Sciences Focus
* 22	Baccalaureate Colleges: Diverse Fields
* 23	Baccalaureate/Associate's Colleges: Mixed Baccalaureate/Associate's
* 24	Special Focus Four-Year: Faith-Related Institutions
* 25	Special Focus Four-Year: Medical Schools & Centers
* 26	Special Focus Four-Year: Other Health Professions Schools
* 27	Special Focus Four-Year: Engineering Schools
* 28	Special Focus Four-Year: Other Technology-Related Schools
* 29	Special Focus Four-Year: Business & Management Schools
* 30	Special Focus Four-Year: Arts, Music & *  Schools
* 31	Special Focus Four-Year: Law Schools
* 32	Special Focus Four-Year: Other Special Focus Institutions
* 33	Tribal Colleges

In [38]:
important_columns = [
    "UNITID", # IPEDS ID
    "HBCU", #historically black college/university
    "HSI ",  #hispanic-serving institution
    "TRIBAL",
    "BASIC2018"
]

carnegie_categories = {}
with codecs.open(data_path + "CCIHE2018-PublicData.csv", encoding='utf-8-sig') as f:
    for row in csv.DictReader(f):
        new_row = {}
        for col in important_columns:
            #strip the new col since HSI has an extra space in the Carnegie data
            #and I don't want to pass on the missing space
            new_row[col.strip()] = row[col] 
        carnegie_categories[row['UNITID']] = new_row

FileNotFoundError: [Errno 2] No such file or directory: '~/IPEDS_DATA_FOLDER/CCIHE2018-PublicData.csv'

### Merge In Carnegie Categories by Row

In [39]:
for record in all_institution_records:
    unitid = record['UNITID']
    if unitid in carnegie_categories.keys():
        carnegie_row = carnegie_categories[unitid]
    else:
        carnegie_row = {"BASIC2018": None, "HBCU":None, "HSI":None, "TRIBAL":None}
    for col in carnegie_row.keys():
        record[col] = carnegie_row[col]

In [40]:
Counter([x['BASIC2018'] for x in all_institution_records])

Counter()

### Diagnostic: print out unique institutions that don't have a Carnegie classification in the dataset

In [41]:
print(set([x['INSTNM'] for x in all_institution_records if x['BASIC2018']  is None]))

set()


# Load and Merge Opportunity Insights Categories

* mrc_table10.csv: [College Level Characteristics from the Opportunity Insights College Scorecard](https://opportunityinsights.org/wp-content/uploads/2018/04/Codebook-MRC-Table-10.pdf)
* [Opportunity insights data page](https://opportunityinsights.org/data/?geographic_level=0&topic=0&paper_id=536#resource-listing)

Note: I wrote 

Columns:
* **super_opeid**: Institution  OPEID  /    Cluster  ID  when  combining  multiple OPEIDs
* **name**: Name of Institution / Super-OPEID Cluster
* **tier**: Selectivity and type combination (constructed from other columns in the dataset):
  * 1 = Ivy Plus (**this is effectively the only category we can merge, due to [the problem of super-OPEIDS](https://robertkelchen.com/2017/08/21/beware-opeids-and-super-opeids/)**)
  * 2 = Other elite schools (public and private)
  * 3 = Highly selective public
  * 4 = Highly selective private
  * 5 = Selective public
  * 6 = Selective private
  * 7 = Nonselective 4-year public
  * 8 = Nonselective 4-year private not-for-profit


In [42]:
important_columns = [
    "super_opeid", # Super OPEID
    "name", #name
    "tier_name", #tier name
    "tier" # the categories of interest
]

op_insights = {}
with codecs.open(data_path + "mrc_table10.csv", encoding='utf-8-sig') as f:
    for row in csv.DictReader(f):
        new_row = {}
        for col in important_columns:
            new_row[col] = row[col]
        op_insights[row['super_opeid']] = new_row
print("{0} opportunity insights categories loaded".format(len(op_insights)))

FileNotFoundError: [Errno 2] No such file or directory: '~/IPEDS_DATA_FOLDER/mrc_table10.csv'

### Merge IvyPlus
This is the one Opportunity Insights category we can use.

#### IvyPlus

In [43]:
## CONFIRM THAT IVYPLUS CAN BE MERGED
tier_code ='1'
op_insights_group = set([x['name'].strip().lower() for x in op_insights.values() if x['tier']==tier_code])
tier_name = [x['tier_name'] for x in op_insights.values() if x['tier']==tier_code][0].lower().replace(" ",".")

print("{0} total {1}\n".format(len(op_insights_group), tier_name))

for name in op_insights_group:
    found = False
    for institution in institutions.values():
        if(institution['INSTNM'].strip().lower() == name):
            found = True
    print("{0} {1}".format(found, name))


IndexError: list index out of range

In [44]:
## MERGE IVYPLUS
op_insights_merged = set()
for row in all_institution_records:
    if row['INSTNM'].strip().lower() in op_insights_group:
        row[tier_name] = 1
        op_insights_merged.add(row['INSTNM'])
    else:
        row[tier_name] = 0
        pass
print("{0} {1} merged".format(len(op_insights_merged), tier_name))
op_insights_merged

NameError: name 'tier_name' is not defined

## Add US News and World Reports Categories
Documentation is available on the [Best Colleges Ranking Category Definition](https://www.usnews.com/education/best-colleges/articles/ranking-category-definitions) guide (last updated Sept 13, 2020. Accessed August 9, 2021). They are essentially supersets of Carnegie Categories.

Region divisions are taken from the [official US Census regions](https://www2.census.gov/geo/pdfs/maps-data/maps/reference/us_regdiv.pdf), and are stored in this repository in `data/us-census-regions.csv`.


In [45]:
## See the key under " Load and Merge Carnegie Categories" 
# for the relationship between BASIC2018 codes 
#  and human-readable Carnegie Categories

usnews_categories ={
    "15": "National Universities",
    "16": "National Universities",
    "18": "Regional Universities",
    "19": "Regional Universities",
    "20": "Regional Universities",
    "21": "National Liberal Arts Colleges",
    "22": "Regional Colleges",
    "23": "Regional Colleges",
    "14": "Regional Colleges",
    "24": "Special Focus Four-Year: Faith-Related Institutions",
    "25": "Special Focus Four-Year: Medical Schools & Centers",
    "26": "Special Focus Four-Year: Other Health Professions Schools",
    "27": "Special Focus Four-Year: Engineering Schools",
    "28": "Special Focus Four-Year: Other Technology-Related Schools",
    "29": "Special Focus Four-Year: Business & Management Schools",
    "30": "Special Focus Four-Year: Arts, Music & * Schools",
    "31": "Special Focus Four-Year: Law Schools",
    "32": "Special Focus Four-Year: Other Special Focus Institutions"
}

for row in all_institution_records:
    if row['BASIC2018'] in usnews_categories:
        row['usnews_category']  = usnews_categories[row['BASIC2018']]
    else:
        row['usnews_category']  = None
        
print("Rows (not institutions) with each category:")
Counter([x['usnews_category'] for x in all_institution_records])

Rows (not institutions) with each category:


Counter()

In [46]:
## Load Census Regions
state_region = {}
with open("data/us-census-regions.csv") as f:
    for row in csv.DictReader(f):
        state_region[row['State Code']] = row['Region']
        
        
## add a few regions to follow the US News and World Report
state_region['PR'] = "South"
state_region['MP'] = "West"
state_region['VI'] = "South"
state_region['GU'] = "West"
state_region['AS'] = "West"

## Apply Census Regions to Regional Institutions, following the US News System 
for row in all_institution_records:
    if (row['usnews_category'] in ['Regional Universities', "Regional Colleges"]):
        if(row['STABBR'] in state_region.keys()):
            row['usnews_category'] += "—" + state_region[row['STABBR']]
        else:
            row['usnews_category'] += "—" + row['STABBR']
        
print("Rows (not institutions) with each category:")
Counter([x['usnews_category'] for x in all_institution_records])       

FileNotFoundError: [Errno 2] No such file or directory: 'data/us-census-regions.csv'

## Add Column for HSI/HBCU/Tribal and IVYPLUS

In [47]:
for record in all_institution_records:
    record['hsi_hbcu_tribal'] = record['HSI'] or record['HBCU'] or record['TRIBAL']
    record['ivy_plus'] = record['ivy.plus']  ==1

# for record in all_institution_records:
#     if record['UNITID'] in institution_pwi.keys():
#         record['pwi'] = institution_pwi[record['UNITID']]>50
#         record['pct_white_2019'] = institution_pwi[record['UNITID']]
#     else:
#         record['pwi'] = None
#         record['pct_white_2019'] = None

# Add Percentages for Every Level
This code computes percentages for every group and subgroup in the dataset.

In [48]:
ranks = ['tenured_full', 'tenured_associate', 'un_tenured_assistant']

## our analysis is only focused on subroups from 2012 onward
## so we only include the demographic characteristics after that time
demo_groups = ['American Indian or Alaska Native',
               'Asian',
               'Black or African American',
               'Hispanic or Latino',
               'Native Hawaiian or Other Pacific Islander',
               'Nonresident alien',
               'Race/ethnicity unknown',
               'Two or more races',
               'White']

## we define the following demographic groups
## as "URM" for this analysis
urm_demo_groups = ['American Indian or Alaska Native',
                   'Black or African American',
                   'Hispanic or Latino',
                   'Native Hawaiian or Other Pacific Islander',
                   'Two or more races']
urm_demo_key    = "black_latino_american_indian_alaska_native_hawaiian_pacific_multiracial"

gender_groups = ['men', 'women']


In [49]:
#institution_row = [x for x in all_institution_records if x['INSTNM'].find("ornell")>-1 and x['year']=='2019-20'][-1]

In [50]:
## ALL SUB-GROUPS ARE RECORDED
## AS A PERCENTAGE OF THE RANK

def set_pct_demo_values(institution_row):
    rank_totals = defaultdict(int)
    
    for rank in ranks:
        rank_key = rank + "_" + "Grand_total"
    #     print("{0}: {1}: {2}".format(
    #         rank_key in institution_row.keys(),
    #         institution_row[rank_key],
    #         rank_key))
    
        rank_totals[rank] = 0

        # for each recorded gender
        for gender in gender_groups:
            rank_gender_key = rank_key + "_" + gender

            if(institution_row[rank_gender_key] is not None):
                pct = (float(institution_row[rank_gender_key]) / institution_row[rank_key])*100.
            else:
                pct = None        

            # convention for gender key: replace _Grand_total and add _pct at the end
            institution_row[rank_gender_key.replace("_Grand_total", "") + "_pct"] = pct

    #         print("{0}: {1}: {2}, {3}".format(
    #             rank_gender_key in institution_row.keys(),
    #             institution_row[rank_gender_key],
    #             rank_gender_key,
    #             rank_gender_key.replace("_Grand_total", "") + "_pct"))
    #         print("    {0}".format(pct))        
    
        # for each demographic group
        for demo in demo_groups:
            demo_key = rank + "_" + demo.replace(" ","_").replace("/","_") + "_total"
            
            if(institution_row[demo_key] is not None):
                pct = (float(institution_row[demo_key]) / institution_row[rank_key])*100.
                
                if(demo in urm_demo_groups):
                    rank_totals[rank] += institution_row[demo_key]
                
            else:
                pct = None

            # convention for demo key: replace "_total" with "_pct"
            institution_row[demo_key.replace("_total", "_pct")] = pct

    #         print("{0}: {1}: {2}".format(
    #             demo_key in institution_row.keys(),
    #             institution_row[demo_key],
    #             demo_key))
    #         print("    {0}".format(pct))

            # for each gender within demographic groups
            for gender in gender_groups:
                demo_gender_key = rank + "_" + demo.replace(" ","_").replace("/","_") + "_" + gender      

                if(institution_row[demo_gender_key] is not None and 
                   institution_row[rank_key] is not None and
                   institution_row[rank_key] != 0):
                    pct = (float(institution_row[demo_gender_key]) / float(institution_row[rank_key]) * 100.)
                else:
                    pct = None

                #convention for demo_gender_key: add _pct at end

                institution_row[demo_gender_key+"_pct"] = pct
    #             print("{0}: {1}: {2}".format(
    #                 demo_gender_key in institution_row.keys(),
    #                 institution_row[demo_gender_key],
    #                 demo_gender_key))
    #             print("    {0}".format(pct))
    
    ## add rank totals
    all_rank_total = 0
    for rank in ranks:
        institution_row[rank + "_" + urm_demo_key + "_total"] = rank_totals[rank]
        all_rank_total += rank_totals[rank]
    institution_row["tenured_un_tenured_" + urm_demo_key + "_total"] = all_rank_total
    institution_row["tenured_un_tenured_" + urm_demo_key + "_pct"] = all_rank_total / row['tenured_un_tenured_Grand_total'] if row['tenured_un_tenured_Grand_total'] else None

In [51]:
## Set demographic values for each institution
for row in all_institution_records:
    set_pct_demo_values(row)

### Validate totals

In [52]:
for record in all_institution_records:
    for rank in ranks:
        rank_key = rank + "_" + "Grand_total"
        calc_total = 0
        for gender in gender_groups:
            rank_gender_key = rank_key + "_" + gender
            if record[rank_gender_key] is not None:
                calc_total += record[rank_gender_key]
        stored_total = 0
        if(record[rank_key] is not None):
            stored_total = record[rank_key]
        
        ## if this runs successfully for every row without an assertion error, then the grand total
        ## is equal to the totals for men and women for every rank.
        assert calc_total == stored_total
print("Validation successful")

Validation successful


# Write to File

In [53]:
pd.DataFrame(all_institution_records).to_csv(data_path + "/" + datetime.datetime.now().strftime("%Y%m%d") + "_all_institution_records.csv",
                                             index=False)

OSError: Cannot save file into a non-existent directory: 'C:\Users\jnajab.AD\IPEDS_DATA_FOLDER'

# Generate Tables for Reporting and Visualization

## Identify Institutions / Rows To Include and Exclude

* Exclude years before 2012-2013

* Institutions that no longer exist or are no longer reporting are omitted from the dataset.

* Institutions reporting two years or fewer are omitted from the dataset.

* Institutions that don't appear in the US News and World Reports classification clusters are omitted.

* Institutions that don't report how many tenured or un-tenured faculty they had in 2019 are omitted from the dataset.

In [54]:
for record in all_institution_records:
    record['year_int']  = int(record['year'][0:4])

all_institution_records[0]['year_int']

IndexError: list index out of range

In [55]:
## include only records since 2012
institution_records = [x for x in all_institution_records if x['year_int']>=2012]
print("Keeping {0} records out of {1}".format(len(institution_records), len(all_institution_records)))


## include only institutions that reported at least three years
institution_counts = Counter([x['UNITID'] for x in institution_records])
insufficient_records =  {k: v for k, v in institution_counts.items() if v <= 2}
print("Omitting {0} records out of {1}".format(len(insufficient_records), len(institution_records)))

#['tenured_full', 'tenured_associate', 'un_tenured_assistant']

## omit institutions that have no recorded data for full, associate, or assistant
no_tenure_records = set([x['UNITID'] for x in institution_records if x['year_int']==2019 and 
                     (x['tenured_full_Grand_total'] is None or
                      x['tenured_associate_Grand_total'] is None or
                      x['un_tenured_assistant_Grand_total'] is None)])

print("Omitting {0} institutions for having no record of full/associate/assistant totals.".format(len(no_tenure_records)))

Keeping 0 records out of 0
Omitting 0 records out of 0
Omitting 0 institutions for having no record of full/associate/assistant totals.


In [70]:
institutions_to_include = [
    x['UNITID'] for x in institution_records if
    x['year_int']  == 2019 and           # exists in 2019
    x['usnews_category'] is not None and # has a US News category
    x['UNITID'] not in insufficient_records.keys() and
    x['UNITID'] not in no_tenure_records
]
print("Included institutions: {0} out of {1}".format(len(institutions_to_include), 
                                                     len(set([x['UNITID'] for x in institution_records]))))

Included institutions: 1250 out of 4852


In [71]:
institution_years = [x for x in institution_records if x['UNITID']  in institutions_to_include]

## Generate Timeseries Tables

Tables need:
* Counts for each category (done)
* Percentages for each category
* Estimate, upper/lower confidence interval for a linear regression for percentages in each category, with the specification:
  * $Y = \beta_0 + \beta_11 \times YEAR + \epsilon$

Summary tables:
* HBCU + HSI + Tribal compared to the rest
* Per-category

Non-summary tables:
* Per-institution

### HBCU + HSI + Tribal compared to the rest

In [72]:
## GENERATE DATASTRUCTURE FOR WHICH KEYS TO DIVIDE BY WHICH

# each column key is the numerator
# with each value the denominator for that key
split_table_cols = {}

for rank in ranks:
    rank_total_key = rank + "_" + "Grand_total"
    split_table_cols[rank_total_key] = rank_total_key

    for gender in gender_groups:
        rank_gender_total_key = rank_total_key + "_" + gender
        #rank_gender_pct_key = rank_gender_total_key.replace("_Grand_total", "") + "_pct"
        split_table_cols[rank_gender_total_key] = rank_total_key

    for demo in demo_groups:
        demo_total_key = rank + "_" + demo.replace(" ","_").replace("/","_") + "_total"
        #demo_pct_key = demo_total_key.replace("_total", "_pct")
        split_table_cols[demo_total_key] = rank_total_key

        for gender in gender_groups:
            demo_gender_total_key = rank + "_" + demo.replace(" ","_").replace("/","_") + "_" + gender      
            #demo_gender_pct_key = demo_gender_total_key + "_pct"
            split_table_cols[demo_gender_total_key] = rank_total_key

In [73]:
set([x for x in split_table_cols.values()])

{'tenured_associate_Grand_total',
 'tenured_full_Grand_total',
 'un_tenured_assistant_Grand_total'}

In [74]:
split_table_cols

{'tenured_full_Grand_total': 'tenured_full_Grand_total',
 'tenured_full_Grand_total_men': 'tenured_full_Grand_total',
 'tenured_full_Grand_total_women': 'tenured_full_Grand_total',
 'tenured_full_American_Indian_or_Alaska_Native_total': 'tenured_full_Grand_total',
 'tenured_full_American_Indian_or_Alaska_Native_men': 'tenured_full_Grand_total',
 'tenured_full_American_Indian_or_Alaska_Native_women': 'tenured_full_Grand_total',
 'tenured_full_Asian_total': 'tenured_full_Grand_total',
 'tenured_full_Asian_men': 'tenured_full_Grand_total',
 'tenured_full_Asian_women': 'tenured_full_Grand_total',
 'tenured_full_Black_or_African_American_total': 'tenured_full_Grand_total',
 'tenured_full_Black_or_African_American_men': 'tenured_full_Grand_total',
 'tenured_full_Black_or_African_American_women': 'tenured_full_Grand_total',
 'tenured_full_Hispanic_or_Latino_total': 'tenured_full_Grand_total',
 'tenured_full_Hispanic_or_Latino_men': 'tenured_full_Grand_total',
 'tenured_full_Hispanic_or_Latino

In [75]:
def year_row():
    return defaultdict(int)

## exclude from split tables any subgroups where the denominator
## of faculty for a given rank is 0. That would mean that no institutions
## reported any faculty in that rank that year

def return_split_table(record_years, split_table_key):        
    included_years = set([x['year_int'] for x in record_years])

    subgroup_table_rows = []

    for subgroup_key in set([x[split_table_key] for x in record_years]):
        row = defaultdict(year_row)        

        subgroup_institution_years = [x for x in record_years if x[split_table_key] == subgroup_key]

        unique_denominator_keys = set([x for x in split_table_cols.values()])

        for institution_year in subgroup_institution_years:
            
            ## TODO: for some reason, this code is double-counting denominators. FIx.
            for denominator_key in unique_denominator_keys:
                if(institution_year[denominator_key] is not None):
                    row[institution_year['year_int']][denominator_key+"_sum"] += institution_year[denominator_key]

            for numerator_key, denominator_key in split_table_cols.items():
                ## only add the numerator if the denominator is not None (likely rare)
                if(institution_year[numerator_key] is not None and institution_year[denominator_key] is not None):
                    row[institution_year['year_int']][numerator_key + "_sum"] += institution_year[numerator_key]
                elif(institution_year[denominator_key] is None and institution_year[numerator_key] is not None):
                    print("denominator key None but numerator exists!!!")

        # patch the tenured and un-tenured sums. TODO: fix the code block starting at line 22
        for year in included_years:
            row[year]['tenured_full_Grand_total_sum'] = row[year]['tenured_full_Grand_total_women_sum'] + row[year]['tenured_full_Grand_total_men_sum']
            row[year]['tenured_associate_Grand_total_sum'] = row[year]['tenured_associate_Grand_total_women_sum'] + row[year]['tenured_associate_Grand_total_men_sum']
            row[year]['un_tenured_assistant_Grand_total_sum'] = row[year]['un_tenured_assistant_Grand_total_women_sum'] + row[year]['un_tenured_assistant_Grand_total_men_sum']
            row[year]['tenured_un_tenured_Grand_total_sum'] = row[year]['tenured_full_Grand_total_sum'] + row[year]['tenured_associate_Grand_total_sum'] + row[year]['un_tenured_assistant_Grand_total_sum']
            
            ## add in joint tenured and un_tenured totals
            ## and joint tenured and un_tenured totals for URM
            total_tt_urm_key = "tenured_un_tenured_" + urm_demo_key + "_total_sum"
            row[year][total_tt_urm_key]  = 0
            for group in urm_demo_groups:
                tf_key  = "tenured_full_" + group.replace(" ","_").replace("/","_") + "_total_sum"
                ta_key  = "tenured_associate_" + group.replace(" ","_").replace("/","_") + "_total_sum"
                ut_key  = "un_tenured_assistant_" + group.replace(" ","_").replace("/","_") + "_total_sum"
                
                row[year][total_tt_urm_key] += row[year][tf_key]
                row[year][total_tt_urm_key] += row[year][ta_key]
                row[year][total_tt_urm_key] += row[year][ut_key]
            
            ## now add pct URM tenure track faculty
            if(row[year]['tenured_un_tenured_Grand_total_sum']):
                row[year]["tenured_un_tenured_" + urm_demo_key + "_total_pct"] = \
                    float(row[year][total_tt_urm_key]) / float(row[year]['tenured_un_tenured_Grand_total_sum']) 
            else:
                row[year]["tenured_un_tenured_" + urm_demo_key + "_total_pct"] = None
                    
        
        for year in included_years:
            
            zero_denominator = False
            
            for numerator_key, denominator_key in split_table_cols.items():
                try:
                    row[year][numerator_key + "_pct"] = (row[year][numerator_key + "_sum"] / row[year][denominator_key + "_sum"])*100.
                except ZeroDivisionError:
                    row[year][numerator_key + "_pct"] = None
                    zero_denominator = True

                    
            row[year]['year'] = year
            row[year]["subgroup"]  = split_table_key
            row[year]["subgroup_value"] = subgroup_key
            
            row[year]['num_institutions'] = len(set([x['UNITID'] for x in subgroup_institution_years if x['year_int'] == year]))
                     
            # skip if there's a zero denominator
            if(zero_denominator):
                print("Zero denominator: {0}, {1}, {2}".format(year, split_table_key, subgroup_key))
            else:
                subgroup_table_rows.append(row[year])
    return subgroup_table_rows

In [76]:
def validate_urm_faculty_columns(split_table_row):
    urm_total = 0
    total_tt_urm_key = "tenured_un_tenured_" + urm_demo_key + "_total_sum"
    
    for group in urm_demo_groups:
        tf_key  = "tenured_full_" + group.replace(" ","_").replace("/","_") + "_total_sum"
        ta_key  = "tenured_associate_" + group.replace(" ","_").replace("/","_") + "_total_sum"
        ut_key  = "un_tenured_assistant_" + group.replace(" ","_").replace("/","_") + "_total_sum"
        urm_total += split_table_row[tf_key]
        urm_total += split_table_row[ta_key]
        urm_total += split_table_row[ut_key]

    assert urm_total == split_table_row[total_tt_urm_key]

### Generate Tables

In [77]:
## confirm that there are only two values
print(set([x['ivy_plus'] for x in institution_years]))

## generate split table, one row for every year
ivy_plus_table = return_split_table(institution_years, 'ivy_plus')

## validate totals
[validate_urm_faculty_columns(x) for x in ivy_plus_table]
pass

{False, True}


In [78]:
validate_urm_faculty_columns(ivy_plus_table[0])

In [79]:
## confirm that there are only two values
print(set([x['hsi_hbcu_tribal'] for x in institution_years]))

## generate split table, one row for every year
hsi_hbcu_tribal_table = return_split_table(institution_years, 'hsi_hbcu_tribal')

{'0', '1'}


In [80]:
## confirm values
print(set([x['usnews_category'] for x in institution_years]))

## generate split table, one row for every year
usnews_category_table = return_split_table(institution_years, 'usnews_category')

{'Special Focus Four-Year: Medical Schools & Centers', 'Regional Universities—West', 'Special Focus Four-Year: Engineering Schools', 'Regional Colleges—Midwest', 'Special Focus Four-Year: Faith-Related Institutions', 'Regional Colleges—South', 'Special Focus Four-Year: Business & Management Schools', 'Special Focus Four-Year: Arts, Music & * Schools', 'Regional Colleges—Northeast', 'Regional Colleges—West', 'Special Focus Four-Year: Other Health Professions Schools', 'Regional Universities—Northeast', 'Regional Universities—Midwest', 'Special Focus Four-Year: Other Special Focus Institutions', 'Special Focus Four-Year: Other Technology-Related Schools', 'Special Focus Four-Year: Law Schools', 'National Liberal Arts Colleges', 'Regional Universities—South', 'National Universities'}
Zero denominator: 2012, usnews_category, Special Focus Four-Year: Business & Management Schools
Zero denominator: 2012, usnews_category, Special Focus Four-Year: Other Technology-Related Schools
Zero denomina

In [84]:
#institution_years[0].keys()

In [86]:
## confirm values for carnegie categories
print(set([x['BASIC2018'] for x in institution_years]))
carnegie_category_table = return_split_table(institution_years, 'BASIC2018')

{'30', '24', '23', '22', '19', '18', '31', '26', '20', '14', '32', '28', '21', '16', '15', '25', '27', '29'}
Zero denominator: 2012, BASIC2018, 28
Zero denominator: 2014, BASIC2018, 28
Zero denominator: 2012, BASIC2018, 29


### Generate Linear Regressions

In [93]:

## the code accepts a custom year key so it can be used
## on the summary tables for collections of institutions
## as well as records for individual institutions
def generate_regression_table(institution_table, year_key = 'year'):
    result_rows = []

    zero_year = 2012

    subgroup_key   = institution_table[0]['subgroup']
    for subgroup_value in list(set([x['subgroup_value'] for x in institution_table])):


        table_df = pd.DataFrame([x for x in institution_table if 
                                          x['subgroup'] == subgroup_key and 
                                          x['subgroup_value'] == subgroup_value])
        table_df['year_num'] = table_df[year_key] - zero_year


        for dv in [x for x in institution_table[0].keys() if x.find("_pct")>-1]:

            formula_text = formula = dv + " ~ year_num"
            mod = smf.ols(formula_text, data=table_df)
            modfit = mod.fit()

            row = {"subgroup"      : subgroup_key,
                   "subgroup_value": subgroup_value,
                   "min_year"      : min([x['year'] for x in institution_table]),
                   "max_year"      : max([x['year'] for x in institution_table]),
                   "dv"            : dv,
                   "formula"       : formula_text,
                   "Intercept"     : modfit.params.Intercept,
                   "year_estimate" : modfit.params.year_num,
                   "year_pvalue"   : modfit.pvalues.year_num,
                   "year_stderr"   : modfit.bse.year_num
                   }
            result_rows.append(row)

    return result_rows

In [94]:
usnews_regression_table = generate_regression_table(usnews_category_table)
ivy_plus_regression_table = generate_regression_table(ivy_plus_table)
hsi_hbcu_tribal_regression_table = generate_regression_table(hsi_hbcu_tribal_table)
carnegie_regression_table = generate_regression_table(carnegie_category_table)


### Write Summary and Regression Tables to Files

In [95]:
pd.DataFrame(usnews_regression_table).to_csv(data_path + "/" + datetime.datetime.now().strftime("%Y%m%d") + 
                                             "_usnews_regression_table.csv",
                                             index=False)
pd.DataFrame(usnews_category_table).to_csv(data_path + "/" + datetime.datetime.now().strftime("%Y%m%d") + 
                                             "_usnews_summary_table.csv",
                                             index=False)

pd.DataFrame(ivy_plus_regression_table).to_csv(data_path + "/" + datetime.datetime.now().strftime("%Y%m%d") + 
                                             "_ivy_plus_regression_table.csv",
                                             index=False)
pd.DataFrame(ivy_plus_table).to_csv(data_path + "/" + datetime.datetime.now().strftime("%Y%m%d") + 
                                             "_ivy_plus_summary_table.csv",
                                             index=False)


pd.DataFrame(hsi_hbcu_tribal_regression_table).to_csv(data_path + "/" + datetime.datetime.now().strftime("%Y%m%d") + 
                                             "_hsi_hbcu_tribal_regression_table.csv",
                                             index=False)
pd.DataFrame(hsi_hbcu_tribal_table).to_csv(data_path + "/" + datetime.datetime.now().strftime("%Y%m%d") + 
                                             "_hsi_hbcu_tribal_summary_table.csv",
                                             index=False)

pd.DataFrame(carnegie_regression_table).to_csv(data_path + "/" + datetime.datetime.now().strftime("%Y%m%d") + 
                                             "_carnegie_regression_table.csv",
                                             index=False)
pd.DataFrame(carnegie_category_table).to_csv(data_path + "/" + datetime.datetime.now().strftime("%Y%m%d") + 
                                             "_carnegie_summary_table.csv",
                                             index=False)

# Produce Regression Tables for Every Institution in the Dataset

In [97]:
counter = 1

institution_regression_table = []
for institution_id in set([x['UNITID'] for x in institution_years]):
    institution_subset_rows = [x for x in institution_years if x['UNITID'] == institution_id]
    for row in institution_subset_rows:
        row['subgroup'] = 'institution'
        row['subgroup_value'] = row['INSTNM']
    regression_rows = generate_regression_table(institution_subset_rows, year_key='year_int')
    for regression in regression_rows:
        regression['carnegie_category'] = institution_subset_rows[0]['BASIC2018']
        regression['usnews_category'] = institution_subset_rows[0]['usnews_category']
        regression['ivy_plus'] = institution_subset_rows[0]['ivy_plus']
        regression['hsi_hbcu_tribal'] = institution_subset_rows[0]['hsi_hbcu_tribal']
    institution_regression_table += regression_rows
    
    if counter % 10 == 0:
        sys.stdout.write(".")
        sys.stdout.flush()
    counter += 1

.............................................................................................................................

In [98]:
#institution_subset_rows[0].keys()

In [99]:
print("Computed {0} regression tables for {1} institutions.".format(len(institution_regression_table), 
                                                                    len(set([x['subgroup_value'] for x in institution_regression_table]))))

Computed 110000 regression tables for 1241 institutions.


### Write regression tables for individual institutions

In [101]:
pd.DataFrame(institution_regression_table).to_csv(data_path + "/" + datetime.datetime.now().strftime("%Y%m%d") + 
                                             "_institution_regression_tables.csv",
                                             index=False)

# Generate Tables of Total Tenured and Tenure Track Faculty for the Article

Documentation from IPEDS201213TablesDoc:
> "Does institution have a tenure system?
> This variable is derived so that users can disaggregate those employees that are nontenured not on tenure track from those employees that are in an institution with no tenure system. Both EAP and Fall Staff components collect these data using 1 data  field which makes it very difficult to produce tables on tenure.  It is assumed that if all employees are reported in the nontenured not on track/no tenure system column/row, then the institution has no tenure system.

In [102]:
tenure_table_cols = {}

for status in ['tenured', 'un_tenured']:
    status_total_key = status + "_" + "Grand_total"
    tenure_table_cols[status_total_key] = status_total_key

    for gender in gender_groups:
        status_gender_total_key = status_total_key + "_" + gender
        #rank_gender_pct_key = rank_gender_total_key.replace("_Grand_total", "") + "_pct"
        tenure_table_cols[status_gender_total_key] = status_total_key

    for demo in demo_groups:
        demo_total_key = status + "_" + demo.replace(" ","_").replace("/","_") + "_total"
        #demo_pct_key = demo_total_key.replace("_total", "_pct")
        tenure_table_cols[demo_total_key] = status_total_key

        for gender in gender_groups:
            demo_gender_total_key = status + "_" + demo.replace(" ","_").replace("/","_") + "_" + gender      
            #demo_gender_pct_key = demo_gender_total_key + "_pct"
            tenure_table_cols[demo_gender_total_key] = status_total_key
            
tenure_table_cols

{'tenured_Grand_total': 'tenured_Grand_total',
 'tenured_Grand_total_men': 'tenured_Grand_total',
 'tenured_Grand_total_women': 'tenured_Grand_total',
 'tenured_American_Indian_or_Alaska_Native_total': 'tenured_Grand_total',
 'tenured_American_Indian_or_Alaska_Native_men': 'tenured_Grand_total',
 'tenured_American_Indian_or_Alaska_Native_women': 'tenured_Grand_total',
 'tenured_Asian_total': 'tenured_Grand_total',
 'tenured_Asian_men': 'tenured_Grand_total',
 'tenured_Asian_women': 'tenured_Grand_total',
 'tenured_Black_or_African_American_total': 'tenured_Grand_total',
 'tenured_Black_or_African_American_men': 'tenured_Grand_total',
 'tenured_Black_or_African_American_women': 'tenured_Grand_total',
 'tenured_Hispanic_or_Latino_total': 'tenured_Grand_total',
 'tenured_Hispanic_or_Latino_men': 'tenured_Grand_total',
 'tenured_Hispanic_or_Latino_women': 'tenured_Grand_total',
 'tenured_Native_Hawaiian_or_Other_Pacific_Islander_total': 'tenured_Grand_total',
 'tenured_Native_Hawaiian_or_O

In [103]:
def return_year_summaries(record_years):    
    record_years = [x for x in all_institution_records if x['year_int']>=2013]
    included_years = set([x['year_int'] for x in record_years])

    subgroup_table_rows = []
    unique_denominator_keys = list(set([x for x in tenure_table_cols.values()]))

    row = defaultdict(year_row)        

    for institution_year in record_years:
        year = institution_year['year_int']

        ## WARNING: THIS CODE IS SOMEHOW PRODUCING 2X THE CORRECT VALUE
        iterations = 0
        for denominator_key in unique_denominator_keys:
            if(institution_year[denominator_key] is not None):
                row[year][denominator_key+"_sum"] += institution_year[denominator_key]
                iterations += 1
        assert iterations <= 2

        for numerator_key, denominator_key in tenure_table_cols.items():
            ## only add the numerator if the denominator is not None (likely rare)
            if(institution_year[numerator_key] is not None and institution_year[denominator_key] is not None):
                row[year][numerator_key + "_sum"] += institution_year[numerator_key]
            elif(institution_year[denominator_key] is None and institution_year[numerator_key] is not None):
                print("denominator key None but numerator exists!!!")

        ## calculate sum of grand totals
#        if('tenured_un_tenured_Grand_total_sum' not in row[year].keys()):
#            row[year]['tenured_un_tenured_Grand_total_sum'] = 0
            
        row[year]['tenured_un_tenured_Grand_total_sum'] += institution_year['tenured_un_tenured_Grand_total']

    # patch the tenured and un-tenured sums. TODO: fix the code block starting at line 13
    for year in included_years:
        row[year]['tenured_Grand_total_sum'] = row[year]['tenured_Grand_total_women_sum'] + row[year]['tenured_Grand_total_men_sum']
        row[year]['un_tenured_Grand_total_sum'] = row[year]['un_tenured_Grand_total_women_sum'] + row[year]['un_tenured_Grand_total_men_sum']
        assert row[year]['tenured_un_tenured_Grand_total_sum'] == row[year]['tenured_Grand_total_sum'] + row[year]['un_tenured_Grand_total_sum']

    for year in included_years:

        zero_denominator = False

        for numerator_key, denominator_key in tenure_table_cols.items():
            try:
                row[year][numerator_key + "_pct"] = (row[year][numerator_key + "_sum"] / row[year][denominator_key + "_sum"])*100.
            except ZeroDivisionError:
                row[year][numerator_key + "_pct"] = None
                zero_denominator = True

        row[year]['year'] = year            
        row[year]['num_institutions'] = len(set([x['UNITID'] for x in record_years if x['year_int'] == year]))

        ## Generate total for tenured + un_tenured
#        row[year]['tenured_un_tenured_Grand_total'] = row[year]['tenured_Grand_total'] + row[year]['un_tenured_Grand_total']
        
        ## Generate demographic percentages for each year
        for demo in demo_groups:
            demo_total = 0
            for status in ['tenured', 'un_tenured']:
                demo_total_key = status + "_" + demo.replace(" ","_").replace("/","_") + "_total_sum"
                demo_total += row[year][demo_total_key]
            
            total_key = 'tenured_un_tenured_' + demo.replace(" ","_").replace("/","_") + "_total_sum"
            row[year][total_key] = demo_total
            row[year][total_key.replace("sum","pct")] = (float(demo_total) / float(row[year]['tenured_un_tenured_Grand_total_sum'])) * 100.
            

        # skip if there's a zero denominator
        if(zero_denominator):
            print("Zero denominator: {0}: {1}".format(year, denominator_key))
        else:
            subgroup_table_rows.append(row[year])
            
    return subgroup_table_rows

In [104]:
year_summary_table = return_year_summaries([x for x in all_institution_records if x['year_int']>=2013])

In [105]:
#[x for x in year_summary_table[0].keys() if x.find("_sum")>-1]
len(year_summary_table[0].keys())

141

### Write table for year summaries

In [106]:
pd.DataFrame(year_summary_table).to_csv(data_path + "/" + datetime.datetime.now().strftime("%Y%m%d") + 
                                             "_year_summary_table.csv",
                                             index=False)